In [ ]:
%pip install --q unstructured langchain langchain-community
%pip install --q "unstructured[all-docs]" ipywidgets tqdm

In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from IPython.display import display as Markdown
from tqdm.autonotebook import tqdm as notebook_tqdm

/var/folders/w2/r1ry0n8s5fd28dmlqqvd6wvm0000gn/T/ipykernel_68660/936963839.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm as notebook_tqdm


In [2]:
local_path = "BMW_X1.pdf"

if local_path:
    loader = PyPDFLoader(file_path=local_path)
    data = loader.load()

In [3]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [4]:
print(pages[3].page_content)

TABLE OF CONTENTS
After a vehicle software update – for example, a Remote Software Up-
grade – the Integrated Owner's Handbook for the vehicle will contain
updated information.
NOTES
Notes .....................................................................................................................  6
QUICK REFERENCE
Getting in ............................................................................................................. 22
Adjustment and operation ...............................................................................  25
On the move ....................................................................................................... 28
CONTROLS
Vehicle cockpit ...................................................................................................  36
Sensors in the vehicle ....................................................................................... 40
Vehicle operating condition ...................................................

In [6]:
#Run this 2 times to remove the index.
pages.remove(pages[3])

In [26]:
#Run this until list index goes out of bound. This will remove the entire appendix
pages.remove(pages[399])

IndexError: list index out of range

In [27]:
clean_pages = pages

In [28]:
print(clean_pages[2])

page_content='WELCOME TO BMW.
Owner's Handbook.
Congratulations on your choice of a BMW.
The better you are acquainted with your vehicle, the easier you will find it
is to operate in traffic. We therefore request:
Please read the Owner's Handbook before setting out in your new BMW.
Also use the Integrated Owner's Handbook in your vehicle. It contains
important information on how to operate your vehicle, helping you get the
most out of your BMW's technical features. It also contains information
to help keep your BMW operating safely on the road and maintain its full
resale value.
When the vehicle leaves the factory, the printed Owner's Handbook is the
most up-to-date version. After a vehicle software update – for example,
a Remote Software Upgrade – the Integrated Owner's Handbook for the
vehicle will contain updated information.
Supplementary information is provided in further on-board literature.
We wish you a safe and pleasant journey.
3
Online Edition for Part no. 01405A7CB30 - II/2

In [29]:
!ollama list

NAME                       ID              SIZE      MODIFIED     
deepseek-r1:1.5b           a42b25d8c10a    1.1 GB    2 months ago    
tinyllama:latest           2644915ede35    637 MB    3 months ago    
nomic-embed-text:latest    0a109f422b47    274 MB    3 months ago    
llava-phi3:3.8b            c7edd7b87593    2.9 GB    3 months ago    
llama3.2:latest            a80c4f17acd5    2.0 GB    5 months ago    


In [30]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [31]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [32]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
chunks = text_splitter.split_documents(data)

In [33]:
#First time, for creating the database

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="bmw-rag",
    persist_directory="./vector_store",
)

In [ ]:
#For simply accessing the persisted database

vector_db = Chroma(
    embedding_function=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="bmw-rag",
    persist_directory="./chroma_db_store",
)

In [34]:
vector_db.get()

{'ids': ['81c4a656-d2ba-4265-90e3-279d3b4974f2',
  '8fb8941f-8dd0-4c36-a95a-a77fdaab53cd',
  '1cfe0dde-d4fd-414d-9cff-469eb6dff16a',
  '6eab68fa-efe6-42aa-97ce-a65534bb04f9',
  '6152c3d0-02d7-434f-88e3-68d0868d7615',
  'a1805296-894b-4c09-8031-04bbce941464',
  'b676d5ae-3eb1-4ea9-9ee6-be5b2e370ff9',
  '8b98ae8a-611e-419b-9678-f23d1245e631',
  'dffa96df-a7e3-4d0c-8e4a-a05ab0dac542',
  '84c97186-af3c-43ba-95b6-e60dc9e7bd3c',
  '3d34df2c-1031-416b-828f-0653044ba3a3',
  'bfd98cfb-dd57-44ae-bf23-9b6968a06a19',
  '0db156c3-a078-44ab-b853-d2b02993ba05',
  '9ed6f25f-bb6d-49e4-9445-ea150bd555cd',
  '661208b8-244b-4dee-a387-08d265fa2ba8',
  '1b6ef1f4-96f5-4d21-8ba8-d4f20880762e',
  'c22814be-b309-4b82-97a7-c06cd1f91fec',
  '226370dc-56ce-4da5-b7e7-a846264076ff',
  'c69a366b-4c38-419a-bb40-4db85dd4096c',
  'd0da0d14-2dc5-4b1c-a511-73b5ddc4ec55',
  'b019efeb-a949-4ac4-a11a-6304fe4464bf',
  'fa1d40c5-99b9-4212-96d4-4398dd949649',
  '7731434a-3692-4b8a-b839-0437ae382f88',
  '46336bfe-37f6-4ac2-9565-

In [35]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from pydantic import BaseModel

In [36]:
local_model = "llama3.2"
llm = ChatOllama(model=local_model)

In [37]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [38]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [39]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
)

In [40]:
chain.invoke("How to activate and deactivate the key card?")

'To activate and deactivate the key card, follow these steps:\n\n1. Park the vehicle safely.\n2. Apply the parking brake.\n3. Turn on the control display.\n4. Consent to Teleservice help (for BMW Accident Assistance).\n5. Select "VEHICLE" /  "Content" in the iDrive menu and choose "Key Card".\n6. Follow the prompts to activate or deactivate the key card.\n\nNote: The exact steps may vary depending on the specific vehicle model and operating system version.'